In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt


print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('train-data.tsv', sep='\t', names=['label', 'message'])
df.head()

# Split into train and test sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
encoder = LabelEncoder()
y_train_bin = encoder.fit_transform(train_data['label'])
y_test_bin = encoder.transform(test_data['label'])

In [ ]:
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

In [ ]:
model.fit(train_data['message'], y_train_bin)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    probas = model.predict_proba([pred_text])[0]
    spam_prob = probas[1]
    threshold = 0.25
    prediction = "spam" if spam_prob > threshold else "ham"
    return [float(spam_prob), prediction]

# Test the function
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)


In [ ]:
def test_predictions():
    test_messages = [
        "how are you doing today",
        "sale today! to stop texts call 98912460324",
        "i dont want to go. can we try it a different day? available sat",
        "our new mobile video service is live. just install on your phone to start watching.",
        "you have won £1000 cash! call to claim your prize.",
        "i'll bring it tomorrow. don't forget the milk.",
        "wow, is your arm alright. that happened to me one time too"
    ]

    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
    passed = True

    for msg, ans in zip(test_messages, test_answers):
        prediction = predict_message(msg)
        print(f"Message: {msg}")
        print(f"Predicted: {prediction[1]}, Expected: {ans}")
        print(f"Spam Probability: {prediction[0]:.4f}\n")
        if prediction[1] != ans:
            passed = False

    if passed:
        print("You passed the challenge. Great job!")
    else:
        print("You haven't passed yet. Keep trying.")

# Run the test
test_predictions()
